In [1]:
# Jannik's models are not SAE Lens, so use Baukit
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# tiny model is in a different direction, add current one to path
import sys
from pathlib import Path
# Get the parent directory of current_dir
parent_dir = str(Path("/root/GroupedSAEs/notebooks/tiny_circuits.ipynb").parent.parent)
sys.path.append(parent_dir)
from tiny_model import TinyModel, tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyModel().to(device)

/root/sae-circuits/circuits2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/sae-circuits/circuits2/lib/python3.11/site-packages/tiny_model/tokenization/tokenization.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.ser

In [2]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
def download_dataset(dataset_name, tokenizer, max_length=256, num_datapoints=None):
    if(num_datapoints):
        split_text = f"train[:{num_datapoints}]"
    else:
        split_text = "train"
    dataset = load_dataset(dataset_name, split=split_text)
    print(dataset)
    total_failed_tokens = 0
    all_tokens = []
    for text in tqdm(dataset["text"]):
        try:
            tokens = [9996] + tokenizer.encode(text)[:max_length]
        except:
            total_failed_tokens += 1
            continue
        # only include if it's at least max_length
        if len(tokens) == max_length+1:
            all_tokens.append(tokens)
    print(f"Failed to tokenize {total_failed_tokens} tokens")
    # convert into a dataset class
    return  Dataset.from_dict({"input_ids": all_tokens})


dataset_name = "noanabeshima/TinyStoriesV2"
max_seq_length = 50
num_datapoints = 3000
# max_seq_length = 100
# num_datapoints = 500
print(f"Downloading {dataset_name}")
dataset = download_dataset(dataset_name, tokenizer=tokenizer, max_length=max_seq_length, num_datapoints=num_datapoints)
true_num_datapoints = len(dataset)
# added BOS
max_seq_length +=1
total_tokens = max_seq_length * true_num_datapoints
print(f"Number of datapoints w/ {max_seq_length} tokens: {true_num_datapoints}")
print(f"Total Tokens: {total_tokens / 1e6}M")


Dataset({
    features: ['text'],
    num_rows: 3000
})


100%|██████████| 3000/3000 [00:04<00:00, 648.04it/s]

Failed to tokenize 0 tokens
Number of datapoints w/ 51 tokens: 3000
Total Tokens: 0.153M


In [3]:
#set torch grad to zero globally
from torch.utils.data import DataLoader
from einops import rearrange
from tqdm import tqdm
from tiny_model import parse_mlp_tag, SparseMLP
N = 1
# activation_names = [f"A{N}", f"T{N}"]
# activation_names = [f"A{N}", f"A{N+1}"]
# activation_names = [f"T{N}", f"T{N+1}"]
activation_names = [f"Rm{N}_S-3_R1_P0", f"T{N}"]
# sae_num = 0
# activation_name = activation_names[sae_num]
file, _, _, _ = parse_mlp_tag(activation_names[0])
sae = SparseMLP.from_pretrained(file).to(device)
# file, _, _, _ = parse_mlp_tag(activation_names[1])
# sae = SparseMLP.from_pretrained(file).to(device)

/root/sae-circuits/circuits2/lib/python3.11/site-packages/tiny_model/sparse_mlp.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [4]:
batch_size = 64
skip_first_n_pos = 0
# def get_dictionary_activations(model, dataset, cache_name, max_seq_length, autoencoder, batch_size=32):
num_features, d_model  = sae.encoder.weight.data.shape
datapoints = dataset.num_rows
all_dictionary_activations_one = torch.zeros((datapoints*max_seq_length, num_features))
all_dictionary_activations_two = torch.zeros((datapoints*max_seq_length, num_features))
both_all_activations = [all_dictionary_activations_one, all_dictionary_activations_two]
token_list = torch.zeros((datapoints*max_seq_length), dtype=torch.int64)


with torch.no_grad(), dataset.formatted_as("pt"):
    dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
    for batch_ind, batch in enumerate(tqdm(dl)):
        batch = batch.to(model.device)
        token_list[batch_ind*batch_size*max_seq_length:(batch_ind+1)*batch_size*max_seq_length] = rearrange(batch, "b s -> (b s)")
        for sae_ind, activation_name in enumerate(activation_names):
            feature_acts = model[activation_name](batch)
            feature_acts[:, :skip_first_n_pos] = 0
            feature_acts = rearrange(feature_acts, "b s n -> (b s) n" )

            both_all_activations[sae_ind][batch_ind*batch_size*max_seq_length:(batch_ind+1)*batch_size*max_seq_length] = feature_acts.cpu()

  0%|          | 0/47 [00:00<?, ?it/s]/root/sae-circuits/circuits2/lib/python3.11/site-packages/tiny_model/sparse_mlp.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st

In [20]:
from interp_utils import *

sae_ind = 0
# sae_ind = 0
num_feature_datapoints = 20
# features = [i for i in range(100)]
# features = [3484, 2094, 3085,  100,  708, 3220,  745, 1756, 2576,  320]
# features = [ 13, 320, 198, 311,  28, 143, 231, 238,  10,  95]
# features = [91]
features = [ 82, 20734,  3233, 6449,  8337,  2734]
# features = [ 9358,  8545,  5924,  9564,  9820,  6745, 10319, 13105,  6576,  6955]
# features = [i for i in range(40)]
# features = [6223, 27334, 28340, 9970, 16493]
# features = [14706, 32670, 10962, 3349, 16162]
# features = [7251]
ablate_context = False
# ablate_context = True
for feature in features:
    nz_ind_amount = both_all_activations[sae_ind][:, feature].count_nonzero()
    print(f"feature: {feature}, non-zero activations: {nz_ind_amount}")
    if(nz_ind_amount == 0):
        continue
    # uniform_indices = get_feature_indices(feature, both_all_activations[sae_ind], k=num_feature_datapoints, setting="uniform")
    uniform_indices = get_feature_indices(feature, both_all_activations[sae_ind], k=num_feature_datapoints, setting="max")
    text_list, full_text, token_list, full_token_list, partial_activations, full_activations = get_feature_datapoints(uniform_indices, both_all_activations[sae_ind][:, feature], tokenizer, max_seq_length, dataset)
    # logit_diffs = ablate_feature_direction(model, full_token_list, activation_name, max_seq_length, ae, feature = feature, batch_size=32, setting="sentences", model_type="causal")
    logit_diffs = None

    html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)
    print(f"feature: {feature}")
    display(HTML(html))
    if(ablate_context):
        all_changed_activations = ablate_context_one_token_at_a_time(model, token_list, activation_name, ae, feature, max_ablation_length=10)
        # html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)

        html = tokens_and_activations_to_html(token_list, all_changed_activations, tokenizer)
        print("Context_ablation\n===============")
        display(HTML(html))

feature: 82, non-zero activations: 15129
feature: 82


feature: 20734, non-zero activations: 0
feature: 3233, non-zero activations: 16
feature: 3233


feature: 6449, non-zero activations: 3
feature: 6449


feature: 8337, non-zero activations: 6
feature: 8337


feature: 2734, non-zero activations: 44
feature: 2734
